# About: 設定ファイルの変更
---

`php.ini`などの設定ファイルを編集して変更内容をコンテナに反映させます。

## 概要

構築したMoodle環境では利用者が変更する可能性のある `php.ini` などの設定ファイルをコンテナではなくVCノードに配置しています。VCノードに配置したファイルはDockerの [bind mount](https://docs.docker.com/storage/bind-mounts/)の機能を利用して、コンテナから参照できるように設定しています。このようにすることで、コンテナをステートレスに保つことができます。

このNotebookでは、コンテナに bind mountしている設定ファイルの内容を変更し、その変更内容をコンテナに反映する手順を示します。

![処理の流れ](images/moodle-030-00.png)

主な手順は以下のようになります。

1. VCノードに配置されている設定ファイルをNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルをVCノードに配置する
5. 設定ファイルの変更を反映させるためにコンテナを再起動する

設定ファイルの編集を行うためには、設定ファイルのパスを指定する必要があります。
Moodleコンテナが参照している設定ファイルの、コンテナ内のパスと、VCノードにおけるパスの対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php.ini</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/php.ini</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/php.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php-zts.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/php-zts.d/</td>
  </tr>  <tr>
    <td style="text-align:left;">/etc/httpd/conf/httpd.conf</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/httpd/conf/httpd.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/httpd/conf.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.modules.d/</td>
    <td style="text-align:left;">/opt/moodle/{container name}/conf/httpd/conf.modules.d/</td>
  </tr>

</table>

> `{container name}` にはコンテナ名が入ります

## パラメータの設定

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `ugroup_name` の値が適切なものかチェックします。

`ugroup_name` に対応する `group_vars` ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

`ugroup_name`に対応する VCノードが実行中であり Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

### コンテナの指定

設定ファイルを変更する対象となるコンテナを指定します。

現在実行中のコンテナの一覧を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose ps --services'

表示されたコンテナ一覧から、対象とするコンテナ名を指定してください。

In [ ]:
# (例)
# target_container = 'moodle-0'

target_container =

### 設定ファイルの指定

変更する設定ファイルのパスを指定してください。ここではコンテナ内のパスを指定してください。

In [ ]:
# (例)
# target_file = '/etc/php.ini'

target_file =

## 設定ファイルの編集

Moodleコンテナの設定ファイルをローカル環境に取得して、Jupyter Notebookの編集機能を用いて設定ファイルを編集します。

![設定ファイルの取得、編集](images/moodle-030-01.png)

次のセルを実行すると、以下の手順が実行されます。

1. VCノードに配置されているMoodleコンテナの設定ファイルをローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

In [ ]:
%run scripts/edit_conf.py
fetch_conf(ugroup_name, target_container, target_file)

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行ってください。

 メニューの[Edit]-[Find]で検索文字を入力することで、編集箇所に移動できます。
> また、ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/{target_fileのファイル名}`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/{target_fileのファイル名}.orig`

次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(ugroup_name, target_container, target_file)

## 編集した設定ファイルの反映

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

![設定ファイルの反映](images/moodle-030-02.png)

### 編集内容の反映

前章で編集した設定ファイルをコンテナに配置して設定を反映させます。

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイルをVCノードに配置する
3. コンテナを再起動して変更した設定ファイルの反映を行う

In [ ]:
apply_conf(ugroup_name, target_container, target_file)

## 変更を取り消す

編集前の設定ファイルの状態に戻します。

![設定ファイル変更の取り消し](images/moodle-030-03.png)

次のセルを実行すると、以下の手順が実行されます。

1. 編集後と編集前の設定ファイルの差分を表示する
2. 編集前の設定ファイルをVCノードに配置する
3. コンテナを再起動して設定値の反映を行う

In [ ]:
revert_conf(ugroup_name, target_container, target_file)